# Sequence classification with Neural Networks
## Overlapping-window RNN model

Overlapping-window RNN model will get as input sliding time windows of a specific pre-defined length from our sequences. Each window will have a label associated with it – **the label of the last timestep in a window**.

Window size is a hyper-parameter that will define how long in the past the model will look. This will depend on the specific task you are trying to solve. 

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import altair as alt

import numpy as np
import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from tmdprimer.datagen import generate_sample, Dataset, Sample

We're going to create a shallow RNN architecture with just one recurrent layer and one output dense unit. But that should be enough for our case given simplicity of our data.

The learning rate is adjusted with a schedule for faster convergence.

In [2]:
import tensorflow as tf

def get_rnn_model():
    rnn_model = tf.keras.Sequential(
        [
            # return_sequences is False now since we are trying to predict last y in a window
            tf.keras.layers.GRU(8, return_sequences=False),
            tf.keras.layers.Dense(1, activation="sigmoid")
        ]
    )
    rnn_model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Nadam(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return rnn_model

The dataset needs to be adapted for this model. So I added the `to_numpy_windows()` and `to_window_tfds()` functions to our classes to produce those sliding windows from our data. Let me know if it's possible to do it using the `window()` function on `tf.data.Dataset`.

In [3]:
dataset = Dataset.generate(train_outlier_prob=0, n_samples=5)
for x, y in dataset.to_window_tfds(window_size=5):
    print(x)
    print(y)
    break

tf.Tensor(
[[0.  ]
 [0.05]
 [0.1 ]
 [0.15]
 [0.2 ]], shape=(5, 1), dtype=float32)
tf.Tensor([0], shape=(1,), dtype=int32)


In [ ]:
data_rnn = []
for window_size in (5, 10, 50):
    for outlier_prob in (0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0):
        print(f"Training for outlier probability: {outlier_prob}, window size: {window_size}")
        dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=100)

        model = get_rnn_model()

        model.fit(
            # we need larger batch sized here since we have many small windows
            x=dataset.to_window_tfds(window_size=window_size).batch(200),
            epochs=5,
            verbose=0
        )
        test_dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=20)
        res = model.evaluate(dataset.to_window_tfds(window_size=window_size).batch(200), verbose=0)
        data_rnn.append({'outlier_prob': outlier_prob, 'accuracy': res[1], 'window_size': window_size})
    
df_rnn = pd.DataFrame(data_rnn)

In [7]:
alt.Chart(df_rnn).mark_line().encode(x='outlier_prob', y='accuracy', color='window_size:N')

alt.Chart(...)

Similar to windowed CNN, RNNs on windows also peform better with larger window sizes at higher outlier probabilities, which makes sense.

Let's see now how the tensorboard graphs look like. You can use those graphs as a reference when comparing them to the more complex models in production.

In [4]:
# Clear any logs from previous runs
from datetime import datetime
!rm -rf ./logs/
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

dataset = Dataset.generate(train_outlier_prob=0.10, n_samples=100)

get_rnn_model().fit(
    x=dataset.to_window_tfds(window_size=20).batch(100),
    epochs=5,
    callbacks=[tensorboard_callback]
)

#%tensorboard --logdir logs/fit

Epoch 1/5
815/815 [==============================] - 17s 18ms/step - loss: 0.4644 - binary_accuracy: 0.7438
Epoch 2/5
815/815 [==============================] - 14s 17ms/step - loss: 0.0785 - binary_accuracy: 0.9793
Epoch 3/5
815/815 [==============================] - 13s 16ms/step - loss: 0.0625 - binary_accuracy: 0.9841
Epoch 4/5
815/815 [==============================] - 14s 17ms/step - loss: 0.0563 - binary_accuracy: 0.9860
Epoch 5/5
815/815 [==============================] - 14s 17ms/step - loss: 0.0525 - binary_accuracy: 0.9870
